Dependencies for Python3 Kernel (RUN THIS TO VIEW IFRAME VISUALIZATIONS):

In [2]:
from IPython.display import HTML, SVG, IFrame

In [15]:
//C kernel test
#include <stdio.h>
#include "/darknet/src/darknet.h"

void main(){printf("hello");}

/tmp/tmpdh6imvnc.c:3:43: fatal error: /data/darknet/include/darknet.h: No such file or directory
compilation terminated.
[C kernel] GCC exited with code 1, the executable will not be executed

# Convolving a Neural Network
---

## Real-time object detection and classification with YOLO

**What you'll learn:** 
- How a Convolutional Neural Network is implemented in a real world application
- How to build convolutional and (max)pooling layers in C
- Run and configure YOLO. (And, if you follow the CUDA/OpenCV installation, run YOLO in real-time with your webcam!)

## Before we get started...

Open Terminal from Jupyter; then copy-paste:

```
curl https://pjreddie.com/media/files/yolo.weights -o /notebooks/sp18/data/darknet/yolo.weights --create-dirs
```

This will download a pre-trained YOLO into your darknet directory. By the time we run it, the weights should be finished downloading.

# 1. Learn what you're working with

Darknet is an elegant neural network framework that can build anything from traditional to recurrent to convolutional to long short-term memory (LSTM) neural networks. For this workshop, we will be focusing on running YOLO by rebuilding functions for the convolutional and pooling layers.


Before we begin, we have to understand the Darknet's architecture. Otherwise, we're just grasping at straws! Below is the abstract structure for a layer. I removed variables that are not in the scope of the workshop to avoid confusion.

In [8]:
#include "/darknet/src/darknet.h"
struct layer_ex{
    LAYER_TYPE type;
    ACTIVATION activation;
    COST_TYPE cost_type;
    void (*forward)   (struct layer, struct network); //Forward propagation
    void (*backward)  (struct layer, struct network); //Backward propagation
    void (*update)    (struct layer, update_args);    //updater
    void (*forward_gpu)   (struct layer, struct network);
    void (*backward_gpu)  (struct layer, struct network);
    void (*update_gpu)    (struct layer, update_args);
    
    int batch_normalize; //boolean -> use batch normalization == 1. There were other normalization methods but we use this
    int batch; // # of samples to be fed in
    
                //vvvv We manipulate these as matrices, but physically, they are 1D arrays w/ calloc
    int inputs;    //data fed into the layer
    int outputs;   //data pushed out of the layer
    int nweights;  // # of weights 
                //^^^^
    int nbiases;    //Additional weights to add

    int h,w,c;  //height h and width w of each input 'matrix', and a count c of all the matrices fed in
    int out_h, out_w, out_c;  //height h and width w of each output 'matrix', and a count c of all the matrices returned
    int n;    //similar to out_c, but it's used for creating the layer
    
    int groups;     //An AlexNet adaption to the convolutional layer. Partitions the kernel (filters)
    int size;       //rank of a given input matrix ()
    int side;
    int stride; //How many columns over the kernel (filter matrix) moves

    float temperature;
    float probability;
    float scale;        //Scale weights with this


    float * biases;      //Memory location for biases we will be considering in the convolutional layer
    float * bias_updates;

    float * scales;       //Memory location for scales for the weights
    float * scale_updates;

    float * weights;      //Memory location for weights
    float * weight_updates;

    float * delta;
    float * output;
    float * squared;
    float * norms;
    
    
//VVVVVVVVVVVVVVVVVVVVVVVVVVVVv
    float * spatial_mean;
    float * mean;
    float * variance;
//                                          Variables for Batch Normalization
    float * mean_delta; 
    float * variance_delta;

    float * rolling_mean;
    float * rolling_variance;

    float * x;
    float * x_norm;
//^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    
    /*CUT GPU STUFF FOR EXAMPLES...*/
};

/tmp/tmp6hab6t_b.c:1:37: fatal error: darknet/include/darknet.h: No such file or directory
compilation terminated.
[C kernel] GCC exited with code 1, the executable will not be executed

# 2. Breaking down the ConvNet pipeline

### Overview

A ConvNet is a series of Convolutional and pooling layers appended to a (usually) fully connected neural network (or ANN.) These layers are needed in order to maintain spatial relations between pixels in an image, something that ANNs are unable to do. Our last workshop went over how to program an ANN from scratch. We now want to show how we can add these additional layers to our neural network.

<img src="assets/images/CNN_pipeline.png"/>

### Convolutional Layer

Recall: For each convolutional layer, we must denote the dimensions of the input. Our inputs will strictly be images, so we have width **w** and height **h** of the input matrix. Since we read the data in as one big array, we partition the array into equal sizes. **c** is the count, as in, how many different matrices we will be reading in.

#### Convolution visualization

The filter (kernel) performs element-wise matrix multiplication in every position that the filter fully "fits" within the input matrix. The filter moves column by column and row by row w.r.t its **stride** (e.g. stride of 2 will move the filter two rows/columns at a time.)

In [2]:
IFrame("assets/cnns/convolution.html", "1000", "500")

**IMPORANT NOTE:** In Darknet, our filters have an additional property: **groups**

What is a group?? Well, let's compare convolution with and without the use of them.

#### Traditional Convolutional Layers vs. Group Convolutional Layers

So a traditional convolution looks something like this. Given a set of samples (left), we apply our filters (middle) to create a set of feature maps.

<img src="assets/cnns/convlayer_traditional.svg"/>

With **groups**, we essentially partition the filters, which allows the convolutional layer to categorize filters.

<img src="assets/cnns/convlayer_group.svg"/>

With this into consideration, we will develop a function to create a convolutional layer.

In [1]:
#include "/darknet/src/darknet.h"
#include "/darknet/src/convolutional_layer.h"

convolutional_layer make_convolutional_layer_ex(int batch, int h, int w, int c, int n, int groups, int size, int stride, int padding, ACTIVATION activation, int batch_normalize, int binary, int xnor, int adam)
{
    
//////////////////////////////////////////////////////////////////////////
    
//Create weights and scales for the weights with respect to Muller Transform

    float scale = sqrt(2./(size*size*c/l.groups));
    for(i = 0; i < l.nweights; ++i) l.weights[i] = scale*rand_normal();
    
    //Calculate the dimensions of the output matrix with this ~~~MAGIC~~~ function! 
    int out_w = convolutional_out_width(l);
    int out_h = convolutional_out_height(l);
    
  
//////////////////////////////////////////////////
    //Batch Normalization

    l.scales = calloc(n, sizeof(float));
    l.scale_updates = calloc(n, sizeof(float));
        for(i = 0; i < n; ++i){
            l.scales[i] = 1;
        }

    l.mean = calloc(n, sizeof(float));
    l.variance = calloc(n, sizeof(float));

    l.mean_delta = calloc(n, sizeof(float));
    l.variance_delta = calloc(n, sizeof(float));

    l.rolling_mean = calloc(n, sizeof(float));
    l.rolling_variance = calloc(n, sizeof(float));
    l.x = calloc(l.batch*l.outputs, sizeof(float));
    l.x_norm = calloc(l.batch*l.outputs, sizeof(float));
   
////////////////////////////////////////////////////////
    //NOT GOING OVER GPU SET UP
///////////////////////////////////////////////////////

    fprintf(stderr, "conv  %5d %2d x%2d /%2d  %4d x%4d x%4d   ->  %4d x%4d x%4d\n", n, size, size, stride, w, h, c, l.out_w, l.out_h, l.out_c);

    return l;
}





In file included from /darknet/src/convolutional_layer.h:4:0,
                 from /tmp/tmpdpyh0hbn.c:2:
/darknet/src/cuda.h:4:21: fatal error: darknet.h: No such file or directory
compilation terminated.
[C kernel] GCC exited with code 1, the executable will not be executed

### Pooling Layer

Now that we have set up the function to create convolutional layers, we must do the same for the pooling layers.

**Recall:** Pooling layers compress the data outputted from the convolutional layer. 

**How:** Input is sectioned into small pieces. One element is chosen from each section and is fed into a smaller, output matrix.

In this workshop, we will focus on **maxpooling**, which chooses the element with the highest value in each section

#### Maxpooling visualization

In [10]:
IFrame("assets/cnns/maxpool.html", 600, 350)

Cool. So now to create the maxpool layer:

In [5]:
#include "/darknet/src/darknet.h"
#include "/darknet/src/maxpool_layer.h"

maxpool_layer make_maxpool_layer_ex(int batch, int h, int w, int c, int size, int stride, int padding)
{
    maxpool_layer l = {0};

    fprintf(stderr, "max          %d x %d / %d  %4d x%4d x%4d   ->  %4d x%4d x%4d\n", size, size, stride, w, h, c, l.out_w, l.out_h, l.out_c);
    return l;
}

/tmp/tmp2ch30_we.c:1:38: fatal error: /darknet/include/darknet.h: No such file or directory
compilation terminated.
[C kernel] GCC exited with code 1, the executable will not be executed

# 3. Build!

Ok, now that we have explained the set up of the convolutional and pooling layers, give a shot at implementing the forward / backward passes of each!

Here are the functions integral to training our ConvNet:
- forward_convolutional_layer() = Applies convolution onto a given convolutional layer

- backward_convolutional_layer() = Applies backpropagation in a given convolutional layer, gets the rate of change of weights


**things to keep in mind:**
 - net.workspace => allocated space for the network data. i.e. TELLS YOU WHICH LAYER YOU ARE CURRENTLY ON and ITS DATA
 - l.batch => # of samples fed into the convolutional layer per run through
 - gemm() = General Matrix Multiplication
 - the layer struct will help guide you through your development

### Convolutional

In [6]:
#include "/darknet/src/darknet.h"
#include "/darknet/src/convolutional_layer.h"

void main()
{
    convolutional_layer l = make_convolutional_layer(1, 5, 5, 3, 2, 1, 5, 2, 1, RELU, 1, 0, 0, 0);
    l.batch_normalize = 1;
    float data[] = {1,1,1,1,1,
        1,1,1,1,1,
        1,1,1,1,1,
        1,1,1,1,1,
        1,1,1,1,1,
        2,2,2,2,2,
        2,2,2,2,2,
        2,2,2,2,2,
        2,2,2,2,2,
        2,2,2,2,2,
        3,3,3,3,3,
        3,3,3,3,3,
        3,3,3,3,3,
        3,3,3,3,3,
        3,3,3,3,3};
    net.input = data;
    forward_convolutional_layer_student(l, net);
    backward_convolutional_layer_student(l, net);
}

void forward_convolutional_layer_student(convolutional_layer l, network net)
{
    int i, j;

    fill_cpu(l.outputs*l.batch, 0, l.output, 1);

    if(l.xnor){
        binarize_weights(l.weights, l.n, l.c/l.groups*l.size*l.size, l.binary_weights);
        swap_binary(&l);
        binarize_cpu(net.input, l.c*l.h*l.w*l.batch, l.binary_input);
        net.input = l.binary_input;
    }

/////////////////////////////////////////////////////////////////////////////////////////
    
    //<><><><><><><><><><><><><><><>
    int m = /*???/l.groups *//// size of output w.r.t the # of groups 
    int k = /*???/l.groups *////input volume size w.r.t the # of groups
    int n = /// output area size

    for(i = 0; i < /*?????*/; ++i){   //For each sample that's fed into this layer...
        for(j = 0; j < /*?????*/; ++j){   //in case of AlexNet implementation, we must account for the groups 
            //configure the memory placement of the pointers
            float *a = // l.???  + l.???*(j/l.groups)         Memory interval for the filter
            float *b = // net.???                             Memory interval for current part of input
            float *c = // l.??? + (i*l.groups + j)*?*?      Memory interval to place outputs

            im2col_cpu(net.input + (i*l.groups + j)*l.c/l.groups*l.h*l.w,  
                l.c/l.groups, l.h, l.w, l.size, l.stride, l.pad, b);   /////GIVEN
            gemm(0,0,m,n,k,1,/*?*/,k,/*?*/,n,1,/*?*/,n); // General Matrix multiplication
            //hint: what applies to k in convolution? what applies to n?
        
        }
    }
///////////////////////////////////////////////////////////////////////////////////////////////////////////
    if(l.batch_normalize){
        forward_batchnorm_layer(l, net);
    } else {
        add_bias(l.output, l.biases, l.batch, l.n, l.out_h*l.out_w);
    }

    activate_array(l.output, l.outputs*l.batch, l.activation);
    if(l.binary || l.xnor) swap_binary(&l);
}

//Backpropagation on a convolutional layer
void backward_convolutional_layer_student(convolutional_layer l, network net)
{
    

    int i, j;
    //REMEMBER THAT IN BACKPROP, THE INPUT AND OUTPUT ARE SWITCHED
    int m = /*???/l.groups *//// size of output w.r.t the # of groups 
    int k = /*???/l.groups *////input volume size w.r.t the # of groups
    int n = /// output area size

    
    gradient_array(l.output, l.outputs*l.batch, l.activation, l.delta); //gets rate of change 

    if(l.batch_normalize){
        backward_batchnorm_layer(l, net);
    } else {
        backward_bias(l.bias_updates, l.delta, l.batch, l.n, k);
    }
    
    
////////////////////////////////////////////////////////////////////////////////////////////////
    //<><><><><><><><><><><>
    for(i = 0; i < /*?????*/; ++i){ //For each sample that's fed into this layer...
        for(j = 0; j < /*?????*/; ++j){ //for each group...
    //<><<><>><><><><><><><>
            float *a = // l.??? + (i*l.groups + j)*?*?          Memory interval for the filter
            float *b = // net.???                             Memory interval for current part of input
            float *c = // l.???  + l.???*(j/l.groups)     Memory interval to place outputs

            float *im = net.input+(i*l.groups + j)*l.c/l.groups*l.h*l.w;
                
            //GIVEN
            im2col_cpu(im, l.c/l.groups, l.h, l.w, 
                    l.size, l.stride, l.pad, b);
            
            gemm(0,1,m,n,k,1,/*?*/,k,/*?*/,k,1,/*?*/,n); //hint: put into same order as in forward_convolutional_layer
            
            //If the change of rate is not 0. hint: remember that 0 = false in C
            if(net./*??????*/){ //update weights.. 
                a = // l.???  + l.???*(j/l.groups) 
                b = // l.??? + (i*l.groups + j)*?*?
                c = // net.???

                gemm(1,0,n,k,m,1,/*?*/,n,/*?*/,k,0,/*?*/,k); //apply the general matrix mult

                col2im_cpu(net.workspace, l.c/l.groups, l.h, l.w, l.size, l.stride, 
                    l.pad, net.delta + (i*l.groups + j)*l.c/l.groups*l.h*l.w);
            }
        }
    }
//////////////////////////////////////////////////////////////////////////////////////////
}

/tmp/tmphd9qknim.c:1:38: fatal error: /darknet/include/darknet.h: No such file or directory
compilation terminated.
[C kernel] GCC exited with code 1, the executable will not be executed

### Maxpooling

Don't worry, maxpooling is much much simpler to create.

In [10]:
#include "/darknet/src/darknet.h"
#include "/darknet/src/maxpool_layer.h"

void main()
{
    //make_maxpool_layer(int batch, int h, int w, int c, int size, int stride, int padding)
    maxpool_layer l = make_maxpool_layer(1, 5, 5, 3, 5, 1, 1);
    float data[] = {1,1,1,1,1,
        1,1,1,1,1,
        1,1,1,1,1,
        1,1,1,1,1,
        1,1,1,1,1,
        2,2,2,2,2,
        2,2,2,2,2,
        2,2,2,2,2,
        2,2,2,2,2,
        2,2,2,2,2,
        3,3,3,3,3,
        3,3,3,3,3,
        3,3,3,3,3,
        3,3,3,3,3,
        3,3,3,3,3};
    net.input = data;
    forward_maxpool_layer_student(l, net);
    backward_maxpool_layer_student(l, net);
}


void forward_maxpool_layer_student(const maxpool_layer l, network net)
{
    int b,i,j,k,m,n;
    int w_offset = -l.pad;
    int h_offset = -l.pad;

    int h = l.out_h;
    int w = l.out_w;
    int c = l.c;
////////////////////////////////////////////////////////////
    
    for(b = 0; b < /*???*/; ++b){  //for each sample fed into layer
        for(k = 0; k < c; ++k){   //For every slice of this sample
            for(i = 0; i < h; ++i){ 
                for(j = 0; j < w; ++j){
                    
                    int out_index = j + w*(i + h*(k + c*b));
                    

                    float max = -FLT_MAX;
                    int max_i = -1;
                    
                    //Find maximum value of each section of given matrix
                    for(n = 0; n < l.size; ++n){
                        for(m = 0; m < l.size; ++m){
                            int cur_h = //height offset + i * l./*?????*/ + n   calculate section w.r.t it's position: 
                            int cur_w = //width  offset + i * l./*?????*/ + m   calculate section w.r.t it's position: 
                            int index = cur_w + l.w*(cur_h + l.h*(k + b*l.c)); 
                            int valid = ( (/*??*/ >= 0) && (/*??*/ < /*??*/) && (/*??*/ >= 0) && (/*??*/ < /*??*/)); //check if within bounds of input matrix
                            float val = (valid != 0) ? net.input[index] : -FLT_MAX; //GET MAX
                            max_i = (val > max) ? index : max_i;
                            max   = (val > max) ? val   : max;
                        }
                    }
   
                    l.output[out_index] = //value of element taken from this section
                    l.indexes[out_index] = //index of element taken
                }
            }
        }
    }
////////////////////////////////////////////////////////////////
}

void backward_maxpool_layer(const maxpool_layer l, network net)
{
    //our layer now has an array of indexes (beacuse of the forward pass)
    int i;
    int h = l.out_h;
    int w = l.out_w;
    int c = l.c;
//////////////////////////////////////////////////////////////////
    //the indexes are fed into this function as a string. how large is this string?
    for(i = 0; i < /*??? x ??? x ??? x ??*/; ++i){ 

        int index = //l./*???*/      pass in maximum element's index
        net.delta[index] += //l./*??*/   rate of change of the i'th element in layer
    }
/////////////////////////////////////////////////////////////////
}

In file included from /darknet/src/image.h:9:0,
                 from /darknet/src/maxpool_layer.h:4,
                 from /tmp/tmpuc0k5fj0.c:2:
/darknet/src/box.h:3:21: fatal error: darknet.h: No such file or directory
compilation terminated.
[C kernel] GCC exited with code 1, the executable will not be executed

# 4. Run!

Now that we have finished writing our functions, let's plug them into the source code!

#### Append code to source files

1. In your folder viewer, go to [PATH TO THIS REPO]/meetings/darknet/src/
2. Copy the inner code of each function and place them in the corresponding C file and function name
    - E.g. In **convolutional_layer.c:** forward_convolutional_layer_student() **->** forward_convolutional_layer()

#### Make Darknet

1. Open up your terminal (or Command Prompt) and **cd** to **[PATH TO THIS REPO]/meetings/darknet/**

2. **make**

#### Run YOLO

In [20]:
//Paste this into your terminal/command prompt
./darknet detect cfg/yolo.cfg yolo.weights data/dog.jpg


//You can mess around with different pictures and the threshold level:
./darknet detect cfg/yolo.cfg yolo.weights data/dog.jpg -thresh 0

//And detect mutiple objects in an image:
./darknet detect cfg/yolo.cfg yolo.weights data/horses.jpg


//You can even detect and classify objects in real-time!
//Follow the guide here to install cuda and OpenCV: https://pjreddie.com/darknet/install/#cuda
// ./darknet detector demo cfg/coco.data cfg/yolo.cfg yolo.weights

/tmp/tmpjjb1cze6.c:2:1: error: expected identifier or '(' before '.' token
 ./darknet detect cfg/yolo.cfg yolo.weights data/dog.jpg
 ^
[C kernel] GCC exited with code 1, the executable will not be executed

# yay u did it